In [85]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
import wget
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [86]:
df_pdfs = pd.read_csv("F:/Environmental Baseline Data/Version 2/Beta deliverables/Index 2 - PDFs.csv")
df_pdfs.head(5)

,Unnamed: 0,file_name,postal,Application filing date,download link,Hearing order,folder,Folder link,Section,Application title,link to folder with ESA,Company Name,Consultant Name,postal upper,Application title short,file size,page count
0,0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,A0H8C0,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C0,Application to Construct and Operate Ekwan Pip...,TBD,TBD
1,1,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,A0H8C1,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C1,Application to Construct and Operate Ekwan Pip...,TBD,TBD
2,2,A0H8C2 - 13.0 EIA - Section 13.7 Wildlife Part 2,A0H8C2,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C2,Application to Construct and Operate Ekwan Pip...,TBD,TBD
3,3,A0H8C3 - 13.0 EIA - Section 13.8 to 13.13,A0H8C3,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C3,Application to Construct and Operate Ekwan Pip...,TBD,TBD
4,4,A0H8C4 - 13.1 App 13A - Alignment Sheets,A0H8C4,2003-03-17,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,GH-1-2003,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.,A0H8C4,Application to Construct and Operate Ekwan Pip...,TBD,TBD


In [1]:
postal_list = df_pdfs["postal"].values
postal_list
len(postal_list)

NameError: name 'df_pdfs' is not defined

# Functions for sorting PDF Chunks

In [95]:
def next_postal_code(strA, strB):
    """
    Takes 2 strings (Postal Codes) as input 
    Reurns True if srtA comes after strB
    Otherwise reurs False
    
    
    Limitation:
    Check only the last 2 digits of the Postal coders to find the next ducument
    
    """
    if strA[:-1] == strB[:-1]:
        if int(strA[-1]) == 1+ int(strB[-1]):
            return True
    if strA[:-2] == strB[:-2]:
        if (strB[-1] == '9') and (strA[-1] == '0'):
            if chr(ord(strB[-2:-1])+1) == strA[-2:-1]:
                return True
    return False
            

In [96]:
def list_string_strip(Postal_List):
    
    """ Takes a list of string (Postal Codes) and perfors these:
             - Strips (removes white spaces) each string of the list 
             - makes sures that all postal characters have 6 alphanumeric characters in upper 
    Postal_List, Flag = list_string_strip(Postal_List)
    """
    Postal_List2 = []
    Flags = []
    for postal in Postal_List:
        postal = postal.strip()
        postal = postal.upper()
        if(len(postal) == 6):
            Postal_List2.append(postal)
        else:
            Flags.append(postal)
    return Postal_List2, Flags
            
Postal_List, Flag = list_string_strip(postal_list)

In [97]:
def generate_PDF_chunks(Postal_List):
    
    """
    Takes a list of string (Postal codes as input)
    and returs a list of list OR list of chunks 
    each chunk represents a list of consecutive postal codes
    """
    chunk_number = 0
    blank_list = []
    List_of_chunks = [[Postal_List[0]]]
    for i in range(1, len(Postal_List)):
        if next_postal_code(Postal_List[i], Postal_List[i-1]):
            List_of_chunks[chunk_number].append(Postal_List[i])
        else:
            chunk_number = chunk_number +1
            List_of_chunks.append(blank_list)
            List_of_chunks[chunk_number].append(Postal_List[i])
    return List_of_chunks
            
        
    

In [100]:
def sorting_PDF_chunks(Postal_List):
    
    # Strips (removes white spaces) each string of the list 
    # and makes sures that all postal characters have 6 alphanumeric characters in upper 
    Postal_List, Flags = list_string_strip(Postal_List)
    
    #Checking if there are any bad Postal Codes
    if(len(Flags) == 0):
        print("All Postal Codes of length 6")
    else: 
        print("Flags exists. List of flagged Postal codes: ")
        for Flag in Flags:
            print(Flag)
        
    #sort the alphanumeric postal codes    
    Postal_List.sort(key = str)
    
    #returs a list of list or list of chunks 
    #each chunk represents a list of strings
    List_of_chunks = generate_PDF_chunks(Postal_List)
    for chunk in List_of_chunks:
        print()
        print("Next Chunk of Postal Codes")
        for postal_code in chunk:
            print(postal_code, end = ",")
        
    retun
        

In [101]:
sorting_PDF_chunks(postal_list)

All Postal Codes of length 6

Next Chunk of Postal Codes
A0H8C0,A0H8C1,A0H8C2,A0H8C3,A0H8C4,A0H8C5,A0H8C6,A0H8C7,A0H8C8,A0H8C9,A0H8D0,A0H8D1,A0H8D2,A0H8D3,A0H8D4,A0H8D5,A0H8D6,A0H8D7,
Next Chunk of Postal Codes
A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

...

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

A0U3G1,A0U3G2,A0Y1T7,A0Y1T8,A0Y1T9,A0Y1U0,A0Y1U1,A0Y1U2,A0Y1U3,A0Y1U4,A0Y1U5,A0Y1U6,A0Y1U7,A0Y1U8,A0Y1U9,A0Y1V0,A0Y1V1,A0Y1V2,A0Y1V3,A0Y1V4,A0Y1V5,A0Y1V6,A0Y1V7,A0Y1V8,A0Y1V9,A0Y1W0,A0Y1W1,A0Z0Y6,A0Z0Y7,A0Z0Y8,A0Z0Y9,A0Z0Z0,A0Z0Z1,A0Z0Z2,A0Z0Z3,A0Z0Z4,A0Z0Z5,A0Z0Z6,A0Z0Z7,A0Z0Z8,A0Z0Z9,A0Z1A0,A0Z1A1,A0Z1A2,A0Z1A3,A0Z1A4,A0Z1A5,A0Z1A6,A0Z1A7,A0Z1A8,A0Z1A9,A0Z1C0,A0Z1C1,A0Z1C2,A0Z1C3,A0Z1C4,A0Z1C5,A0Z4I1,A0Z4I2,A0Z4I3,A0Z4I4,A1C2E2,A1C2E5,A1C2E6,A1C2E7,A1C2E8,A1C2E9,A1C2F0,A1C3V5,A1C3V6,A1C3V7,A1C3V8,A1C3V9,A1C3W0,A1C3W1,A1C3W2,A1C3W3,A1C3W4,A1C3W5,A1C3W6,A1C3W7,A1C3W8,A1C3W9,A1C3X0,A1C3X1,A1C3X2,A1C3X3,A1C3X4,A1C3X5,A1C3X6,A1C3X7,A1C3X8,A1C3X9,A1C3Y0,A1C3Y1,A1C3Y2,A1C3Y3,A1C3Y4,A1C3Y5,A1C3Y6,A1C3Y7,A1C3Y8,A1C3Y9,A1C3Z0,A1C3Z1,A1C3Z2,A1C3Z3,A1C3Z4,A1C3Z5,A1D5R4,A1D5R5,A1D5R6,A1D5R7,A1D5R8,A1D5R9,A1D5S0,A1D5S1,A1D5S2,A1D5S3,A1D5S4,A1D5S5,A1D5S6,A1D5S7,A1D5S8,A1D5S9,A1I9U0,A1I9U1,A1I9U2,A1I9U3,A1I9U4,A1I9U5,A1I9U6,A1I9U7,A1I9U8,A1I9U9,A1I9V0,A1I9V1,A1I9V2,A1I9V3,A1I9V4,A1I9V5,A1I9V6,A1I9V7

I just realized that we might not even need to worry about making chunks. The documents that are associated with a particular particular project need to be treated as one chunk. We only need to sort these PDF files. 

# Finding the PDFs in a sequesnce 

In [103]:
application_list = df_pdfs["Application title"].values
application_set = set(application_list)
len(application_set)

36

In [ ]:
for application in application_set:
    